In [1]:
import torch
import numpy as np
import scipy.integrate
solver = scipy.integrate.solve_ivp
from numpy import sin, cos 
import matplotlib.pyplot as plt
import random


seed = 12
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.determinstic = True

In [2]:
#parameters
a11 = 1.0
a12 = -1.0
a21 = -1.0
a22 = 1.0

def func(t, state):
  dvdt = np.zeros_like(state)
  dvdt[0] = a11*state[0] + a12*state[0]*state[1]
  dvdt[1] = a21*state[1] + a22*state[0]*state[1]
  return dvdt

#　the time grid
M = 100
tend = 5.0
t_eval = np.linspace(0,tend,M)
dt = t_eval[1]-t_eval[0]
num = 1000
#state = np.random.rand(num, 2)
state = np.random.uniform(low=0.0, high=1.0, size=(num,2))
flag = False
for i in range(num):
  sol = solver(func, [0, tend], state[i], t_eval=t_eval)
  tval = sol['t']
  q1 = sol['y'][0]
  q2 = sol['y'][1]
  xval = np.vstack([q1,q2])
  dxdt = func(tval, xval)
  if flag:
    x_input = np.concatenate([x_input, xval], 1)
    x_target = np.concatenate([x_target, dxdt], 1)
  else:
    x_input = xval
    x_target = dxdt
    flag = True

x_target_max = np.abs(x_target).max(axis=1)
A = np.diag(x_target_max)
B = A
    
x_input = np.matmul(np.linalg.inv(B),x_input)
x_target = np.linalg.inv(A)@x_target

In [ ]:
target_file = np.savetxt("target.csv", x_target.T, delimiter=',')
input_file = np.savetxt("input.csv", x_input.T, delimiter=',')

In [4]:
A_file = np.savetxt("A.csv", A, delimiter=',')
B_file = np.savetxt("B.csv", B, delimiter=',')